🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

## Creating custom components that incorporate threading and logging

To create a custom component in Haystack that adheres to the requirements for custom components and incorporates global state management, let's design a SessionTracker component. This component will track user sessions and demonstrate thread safety and state hygiene. The SessionTracker will use a class-level dictionary to store session data, ensuring thread-safe operations with a lock and providing a method to clean outdated session information.

Step 1: Import Required Libraries

In [4]:
from haystack import component
import logging
from threading import Lock

# Initialize logging for debugging and operational insight
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


Step 2: Define the SessionTracker Component
This component will manage user sessions, including creating new sessions and updating existing ones. It will use a class-level dictionary to store session states and a lock to ensure thread-safe modifications.

In [5]:
@component
class SessionTracker():
    _sessions = {}  # Class-level dictionary to store session data
    _lock = Lock()  # Lock for thread-safe operations on _sessions

    @component.output_types(session_data=str, status=str)
    def run(self, user_id: str, action: str) -> dict:
        """
        Updates or initializes the session state for a given user based on the action performed.
        """
        with SessionTracker._lock:
            if user_id not in SessionTracker._sessions:
                # Initialize session if not existing
                SessionTracker._sessions[user_id] = "Session started"
                logger.info(f"Session started for user: {user_id}")
            else:
                # Update session based on action
                SessionTracker._sessions[user_id] += f", {action}"
                logger.info(f"Session updated for user: {user_id} with action: {action}")

            return {
                "session_data": SessionTracker._sessions[user_id],
                "status": "success"
            }

    @classmethod
    def clean_sessions(cls):
        """
        Cleans up outdated or inactive session data to maintain state hygiene.
        """
        with cls._lock:
            # This is a placeholder for actual logic to identify and clean outdated sessions
            cls._sessions.clear()
            logger.info("All sessions cleared.")


Step 3: Implementing and Using the Component
This component can be integrated into a Haystack pipeline to track user sessions dynamically. Below is an example of how you might use the SessionTracker in a standalone manner, demonstrating starting sessions and performing actions.

In [6]:
if __name__ == "__main__":
    session_tracker = SessionTracker()

    # Simulate starting a session and performing actions for users
    users = ["user1", "user2"]
    actions = ["login", "search", "logout"]
    
    for user in users:
        for action in actions:
            result = session_tracker.run(user_id=user, action=action)
            logger.info(f"Action: {action}, Result: {result}")

    # Clean sessions periodically or based on specific triggers
    SessionTracker.clean_sessions()


INFO:__main__:Session started for user: user1
INFO:__main__:Action: login, Result: {'session_data': 'Session started', 'status': 'success'}
INFO:__main__:Session updated for user: user1 with action: search
INFO:__main__:Action: search, Result: {'session_data': 'Session started, search', 'status': 'success'}
INFO:__main__:Session updated for user: user1 with action: logout
INFO:__main__:Action: logout, Result: {'session_data': 'Session started, search, logout', 'status': 'success'}
INFO:__main__:Session started for user: user2
INFO:__main__:Action: login, Result: {'session_data': 'Session started', 'status': 'success'}
INFO:__main__:Session updated for user: user2 with action: search
INFO:__main__:Action: search, Result: {'session_data': 'Session started, search', 'status': 'success'}
INFO:__main__:Session updated for user: user2 with action: logout
INFO:__main__:Action: logout, Result: {'session_data': 'Session started, search, logout', 'status': 'success'}
INFO:__main__:All sessions c